In [1]:
# Dependencies
import os
import pandas as pd

# set spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-14 23:40:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-08-14 23:40:55 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [5]:
# Count the number of records (rows) in the dataset
df.count()

10319090

In [6]:
# Drop NaN values
df = df.dropna(how="any")

In [7]:
# Create review_id_table dataframe
review_id_table_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

In [8]:
# Create review_id_table dataframe
review_id_table_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

In [9]:
# Create products_table dataframe
products_table_df = df.select(["product_id", "product_title"])
products_table_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439873800|There Was an Old ...|
|1623953553|      I Saw a Friend|
|142151981X|Black Lagoon, Vol. 6|
|014241543X|           If I Stay|
|1604600527|Stars 'N Strips F...|
|0399170863|            The Liar|
|1517007240|Devil in the Deta...|
|0671728725|Knowing When to S...|
|1111349533|The American Pageant|
|812211637X|Punjabi C.L. Bibl...|
|0679413359|The Aeneid (Every...|
|0983990662|The YMCA of Middl...|
|156852062X|Memoirs By Harry ...|
|B0007E4WJ4|Arms of our fight...|
|0664254969|Presbyterian Cree...|
|1570913722|Hello Ocean/Hola mar|
|0345313860|The Vampire Lesta...|
|0670025593|The Book of Life ...|
|161893158X|Disney Descendant...|
|007179512X|CompTIA A+ Certif...|
+----------+--------------------+
only showing top 20 rows



In [13]:
# Create customers_table dataframe
customers_table_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_table_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37119503|             3|
|    3180632|             1|
|   14335758|             6|
|   44892935|             1|
|     740955|             1|
|   20228001|             4|
|   23810736|             1|
|   51291733|             1|
|   10632307|             1|
|   40758291|            21|
|   30142445|             2|
|   17356431|             8|
|   49948219|             3|
|   47840769|             2|
|   41826621|             2|
|   15073256|             1|
|   43801871|             5|
|    3223564|             1|
|   29191691|            22|
|   51052113|            79|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create vine_table dataframe
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RJOVP071AVAJO|          5|            0|          0|   N|
|R1ORGBETCDW3AI|          5|            0|          0|   N|
| R7TNRFQAOUTX5|          5|            0|          0|   N|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|
|R13U5PBJI1H94K|          2|            1|          1|   N|
|R1H8UVH990F8VE|          5|            2|          2|   N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|
| R2NO2HXK16Y4J|          5|            0|          0|   N|
|R245YIAVJK82ZL|          5|            0|          0|   N|
| RFBMLBWQOZ1UM|          5|            0|          0|   N|
| RHE1FEWV3KTQH|          5|            0|          0|   N|
|R2BB9U7PHO7ETO|          5|            0|          1|   N|
|R28XJJP754GYXO|          4|            

In [15]:
# Postgres setup
# Configure settings for RDS
mode = "append"
jdbc_url = "jdbc:postgresql://amazonbookreviews.c0xjlfeovn0x.us-west-2.rds.amazonaws.com:5432/amazonbookreviews"
config = {"user":"<postgres username>", 
          "password":"<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataframe to review_id_table in RDS
review_id_table_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to products_table in RDS
products_table_df.write.jdbc(url=jdbc_url, table="products_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to customers_table in RDS
customers_table_df.write.jdbc(url=jdbc_url, table="customers_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to vine_table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)